In [1]:
import requests
import json
from openai import AzureOpenAI
import os

"""
IN CMD:

setx OPENAI_KEY_ABB ""
setx OPENAI_BASE_ABB ""

setx AISEARCH_KEY_ABB ""
setx AISEARCH_BASE_ABB ""

setx DOCUMENT_INTELLIGENCE_KEY_ABB ""
setx DOCUMENT_INTELLIGENCE_BASE_ABB ""
"""

openai_api_version = "2024-03-01-preview"
openai_key = os.environ.get("OPENAI_KEY_ABB")
openai_endpoint = os.environ.get("OPENAI_BASE_ABB")

index_name = "ai_workshop"
aisearch_key = os.environ.get("AISEARCH_KEY_ABB")
aisearch_endpoint = os.environ.get("AISEARCH_BASE_ABB")

openai_api_version = "2024-03-01-preview"
openai_key = os.environ.get("OPENAI_KEY_ABB")
openai_endpoint = os.environ.get("OPENAI_BASE_ABB")

client = AzureOpenAI(
  api_version = openai_api_version,
  api_key = openai_key, 
  azure_endpoint = openai_endpoint
)

### Simple Hybrid query

In [4]:
question = "What are prerequisite for commissioning?"

response = client.embeddings.create(
    input = question,
    model= "text-embedding-ada-002"
)

# Define the URL and headers
url = f'{aisearch_endpoint}/indexes/{index_name}/docs/search?api-version=2023-11-01'
headers = {
    'Content-Type': 'application/json',
    'api-key': aisearch_key
}

# Define the request body
payload = {
    "vectorQueries": [
        {
            "vector": response.data[0].embedding,
            "fields": "contentVector",
            "kind": "vector",
            "k": 50
        }
    ],
    "search": question,
    "top": "5"
}

# Convert payload to JSON
data = json.dumps(payload)

# Make the POST request
response_index = requests.post(url, headers=headers, data=data)

# Check if the request was successful
if response_index.status_code == 200:
   
    for item in response_index.json()['value']:
        print(f"Title: {item['title']}")
        print(f"Paragraph: {item['paragraph']}")
        print(f"Content: {item['content']}")
        print(f"Search Score: {item['@search.score']}")
        print()


else:
    print("Request failed with status code:", response_index.status_code)
    print(response_index.text)

Title: machinery manual sample
Paragraph: 2.5.1. Prerequisite for commissioning/startup
Content: 
Before commissioning/starting up the pump set, make sure that the following conditions are met:

. The pump set has been properly connected to the electric power supply and is equipped with all protection devices.

· The pump has been flooded up to the specified minimum water level. (See general arrangement drawing).

· The direction of rotation has been checked.

· All auxiliary connections required are connected and operational.

· The transport lock has been removed.

· The lubricants have been checked and filled in.

· After prolonged shutdown of the pump (set), the required activities have been carried out.

· The coupling alignment has been checked.


Search Score: 0.03333333507180214

Title: machinery manual sample
Paragraph: 3.4.1. Start-up
Content: 
When commissioning pump in new borehole, initially only operate the pump for approximately 10 minutes with the shut-off element sligh

### Simple Hybrid query with filters on metadata

In [6]:
question = "What are prerequisite for commissioning?"

response = client.embeddings.create(
    input = question,
    model= "text-embedding-ada-002"
)

# Define the URL and headers
url = f'{aisearch_endpoint}/indexes/{index_name}/docs/search?api-version=2023-11-01'
headers = {
    'Content-Type': 'application/json',
    'api-key': aisearch_key
}

# Define the request body
payload = {
    "vectorQueries": [
        {
            "vector": response.data[0].embedding,
            "fields": "contentVector",
            "kind": "vector",
            "k": 50
        }
    ],
    "search": question,
    "filter": "paragraph eq '2.5.1. Prerequisite for commissioning/startup'",
    "top": "5"
}

# Convert payload to JSON
data = json.dumps(payload)

# Make the POST request
response_index = requests.post(url, headers=headers, data=data)

# Check if the request was successful
if response_index.status_code == 200:
   
    for item in response_index.json()['value']:
        print(f"Title: {item['title']}")
        print(f"Paragraph: {item['paragraph']}")
        print(f"Content: {item['content']}")
        print(f"Search Score: {item['@search.score']}")
        print()


else:
    print("Request failed with status code:", response_index.status_code)
    print(response_index.text)

Title: machinery manual sample
Paragraph: 2.5.1. Prerequisite for commissioning/startup
Content: 
Before commissioning/starting up the pump set, make sure that the following conditions are met:

. The pump set has been properly connected to the electric power supply and is equipped with all protection devices.

· The pump has been flooded up to the specified minimum water level. (See general arrangement drawing).

· The direction of rotation has been checked.

· All auxiliary connections required are connected and operational.

· The transport lock has been removed.

· The lubricants have been checked and filled in.

· After prolonged shutdown of the pump (set), the required activities have been carried out.

· The coupling alignment has been checked.


Search Score: 0.03333333507180214



# Hybrid query with semantic ranking

In [10]:
question = "What are prerequisite for commissioning?"

response = client.embeddings.create(
    input = question,
    model= "text-embedding-ada-002"
)

# Define the URL and headers
url = f'{aisearch_endpoint}/indexes/{index_name}/docs/search?api-version=2023-11-01'
headers = {
    'Content-Type': 'application/json',
    'api-key': aisearch_key
}

# Define the request body
payload = {
    "vectorQueries": [
        {
            "vector": response.data[0].embedding,
            "fields": "contentVector",
            "kind": "vector",
            "k": 50
        }
    ],
    "search": question,
    "queryType": "semantic",
    "semanticConfiguration": "my-semantic-config",
    #"select": "title, product, paragraph",
    "captions": "extractive",
    #"answers": "extractive",
    #"filter": "category eq 'Databases'",
    #"vectorFilterMode": "postFilter",
    "top": "5"
}

# Convert payload to JSON
data = json.dumps(payload)

# Make the POST request
response_index = requests.post(url, headers=headers, data=data)

# Check if the request was successful
if response_index.status_code == 200:
   
    for item in response_index.json()['value']:
        print(f"Title: {item['title']}")
        print(f"Paragraph: {item['paragraph']}")
        print(f"Content: {item['content']}")
        print(f"Search Score: {item['@search.score']}")
        print()


else:
    print("Request failed with status code:", response_index.status_code)
    print(response_index.text)

Title: machinery manual sample
Paragraph: 2.5.1. Prerequisite for commissioning/startup
Content: 
Before commissioning/starting up the pump set, make sure that the following conditions are met:

. The pump set has been properly connected to the electric power supply and is equipped with all protection devices.

· The pump has been flooded up to the specified minimum water level. (See general arrangement drawing).

· The direction of rotation has been checked.

· All auxiliary connections required are connected and operational.

· The transport lock has been removed.

· The lubricants have been checked and filled in.

· After prolonged shutdown of the pump (set), the required activities have been carried out.

· The coupling alignment has been checked.


Search Score: 0.03333333507180214

Title: machinery manual sample
Paragraph: 3.4. Commissioning/start-up
Content: 

Search Score: 0.023037323728203773

Title: machinery manual sample
Paragraph: 2.5. Commissioning/Startup/Shutdown
Conten